Please **submit this Jupyter notebook through Canvas** no later than **Monday November 12, 12:59**, before the start of the lecture.

Homework is in **groups of two**, and you are expected to hand in original work. Work that is copied from another group will not be accepted.

# Exercise 0
Write down the names + student ID of the people in your group.

**Jordan Earle** (12297127)

**Natasja Wezel** (11027649)

-----
# Exercise 1

## (a)
Let $A$ be the matrix $\begin{bmatrix} 1 & -1 & \alpha \\ 2 & 2 & 1 \\ 0 & \alpha & -3/2 \end{bmatrix}$. For which values of $\alpha$ is $A$ singular?

In [1]:
import numpy as np
import scipy as sp
from scipy import linalg as la
from sympy import *
from sympy import Matrix
x = Symbol('x')
matA = Matrix([[1, -1, x], [2, 2, 1], [0, x, -3/2]])

print(matA)

print(matA.det())
equation = matA.det()
solutions = solve(equation)

print("The values where alpha is zero are:", solutions[0],solutions[1])

Matrix([[1, -1, x], [2, 2, 1], [0, x, -1.50000000000000]])
2*x**2 - x - 6.0
The values where alpha is zero are: -1.50000000000000 2.00000000000000


## (b)
Consider the following linear system of equations:
$$
\begin{align*}
2x + y + z & = 3 \\
2x - y + 3z &= 5 \\
-2x + \alpha y + 3z &= 1.
\end{align*}
$$
For what values of $\alpha$ does this system have an infinite number of solutions?

In [2]:
a = Symbol('a')

matB = Matrix([[2, 1, 1], [2, -2, 3], [-2, a, 3]])

print(matB.det())
equation = matB.det()
solutions = solve(equation)

print("The values where alpha causes the system to be singular and therefore infinitly many solutions are: ", solutions[0])

-4*a - 28
The values where alpha causes the system to be singular and therefore infinitly many solutions are:  -7


## (c)
Denote the columns of an $n \times n$ matrix $A$ as $A_k$ for $k=1,\ldots,n$. We define the function $||A||_* = \max_k ||A_k||_2$. Show that $||A||_*$ is a norm, in that it satisfies the first three properties of a matrix norm (cf. §2.3.2).

The first three properties of a matrix norm, where A and B are any matrices, are:
1. Positivity:

||A|| > 0 if A $\neq$ $\textbf{O}$

2. Homogeneity:

||$\lambda \textbf{A} $|| = $|\lambda| \cdot || \textbf{A} ||$ for any scalar $\lambda$

3. Triangle inequality:

$||\textbf{A} + \textbf{B}|| \leq  || \textbf{A} || + || \textbf{B} ||$

Note that the first two properties together imply that $||A|| = 0$ if, and only if, $A = 0$.

For all proves the fact that $max_k ||A_k||_2$ is a norm is used.

$$ % TODO: I'm not sure how to properly write this $$

1.To prove that $||A||_*$ is positive

$$
\begin{align}
    max_{1 \leq k \leq n} || A_k ||_2 > 0
\end{align}
$$
$ $

$\exists k : || A_k ||_2 > 0 $, so: $A_k \neq 0 => A \neq 0$

This proves that the first part:
$||A||_*$ > if $A \neq O$

2. To prove the second property:

$$
\begin{align*}
    || \lambda A ||_* &= max_k ||\lambda A_k||_2\\
                      &= max_k (|\lambda | \cdot ||A_k||_2)\\
                      &= |\lambda| \cdot max_k ||A_k||_2\\
                      &= |\lambda| \cdot ||A||_*
\end{align*}
$$

3. To prove the third property:

$$
\begin{align*}
    || A + B ||_* = max_k || A_k + B_k||_2 &\leq max_k (||A_k||_2 + ||A_k||_2) \\ 
                      &\leq max_k ||A_k|| + max_l ||B_l|| \\
                      &\leq ||A||_* + ||B||_*|
\end{align*}
$$
 

----
# Exercise 2
For solving linear systems such as $Ax = b$, it is unnecessary (and often unstable) to compute the inverse $A^{-1}$. Nonetheless, there can be situations where it is useful to compute $A^{-1}$ explicitly. One way to do so is by using the LU-decomposition of $A$.

## (a)
Write an algorithm to compute $A^{-1}$ for a non-singular matrix $A$ using its LU-decomposition. You can use `scipy.linalg.lu` (which returns an LU-decomposition with _partial pivoting_, i.e., with a permutation matrix $P$) and the other `scipy.linalg.lu_*` functions, but not `scipy.linalg.inv` (or other methods for computing matrix inverses directly).

In [55]:
#Remeber that when accessing a matrix, Matrix[row][column] & Matrix[m][n]

import numpy as np

def solveInverse(matA):

    p, l, u = sp.linalg.lu(matA)
    
    #print(p)
    #print(l)
    #print(u)
    #print(l[2][1])
    shapeA = matA.shape
    validity = True
    #print(p.dot(l.dot(u)))
    
    
    for i in range(0,shapeA[0]):
        if l[i][i] == 0 or u[i][i] == 0:
            print("Matrix is not invertable!")
            return False, np.empty(shape = s)
        
    b = p.transpose().dot(np.eye(shapeA[0], dtype=float))
    #print(b)
    c = forwardSublower(shapeA, l, b)
    #print(c)
    inverted = backwardSubsolver(shapeA, u, c)
    
    return True, inverted

def forwardSublower(s,lw,solM):
    tmpMat = np.empty(shape = s)
    for n in range(0, s[1]): # cycle through columns
        #print("In column: ",n)
        for m in range(0, s[0]): # cycle through rows
            #print("The value being used to solve for x is", solM[m][n])
            tmpMat[m][n] = (solM[m][n]/lw[m][m]) # Find current X val 
            #print("The current solution for the matrix is: ", tmpMat[m][n])
            for i in range(m+1, s[1]): # Subtract that value from all other colutions
                #print("The current value of the solution matrix is", solM[i][n])
                #print("The current value of the x matrix is", tmpMat[m][n])
                #print("The current value of the lower matrix is", lw[i][n])
                solM[i][n] = solM[i][n] - (lw[i][m]*tmpMat[m][n])
    return tmpMat

def backwardSubsolver(s, up, solM):
    tmpMat = np.empty(shape = s)
    for n in range(s[0]-1, -1, -1):#starting at the last column, go through the columns
        for m in range(s[1]-1, -1, -1): # starting at the last row, go through the rows.
            tmpMat[m][n] = (solM[m][n]/up[m][m]) # Find current X val 
            for i in range(m-1, -1, -1): # Subtract that value from all other colutions
                 solM[i][n] = solM[i][n] - (up[i][m]*tmpMat[m][n])
    return tmpMat

    
MatA = np.array([[5., 7., 12.], [13.,14.,2.], [11., 15., 10]])
#print(MatA)
#Somethinng weird is happening here and its pssing me off.  Check with cornelis at some point
# p, l, u = sp.linalg.lu(matA)
# print(p)
# print(l)
# print(u)
# print(p.dot(l.dot(u)))

check, inverted = solveInverse(MatA)
if check:
    print(inverted-sp.linalg.inv(MatA))
    print(inverted.dot(MatA))

[[-1.66533454e-16 -1.11022302e-16  1.11022302e-16]
 [ 0.00000000e+00  5.55111512e-17  0.00000000e+00]
 [ 0.00000000e+00  6.07153217e-18 -1.38777878e-17]]
[[ 1.00000000e+00 -1.55431223e-15 -2.22044605e-15]
 [-1.11022302e-16  1.00000000e+00  1.11022302e-15]
 [-6.93889390e-17 -1.24900090e-16  1.00000000e+00]]


## (b)
What is the computational complexity of your algorithm, given that the input matrix has size $n \times n$?

It will have a computational complexity of $n^{3}/3$

## (c)
Apply your Python code to compute the inverse of the Hilbert matrix $H_n$ for $n=1, \ldots, 12$ (see https://en.wikipedia.org/wiki/Hilbert_matrix) -- you can use `scipy.linalg.hilbert`. This matrix is _very_ ill-conditioned, so computing its inverse is very hard for large $n$.

Compare the inverse with the "true" inverse given by `scipy.linalg.invhilbert`. Output a (`plt.semilogy`) graph showing how the $\infty$-norm of their difference progresses for $n$.

In [56]:
import scipy as sp
from scipy import linalg as la

for n in range(12):
    # get the Hilbert matrix
    Hn = la.hilbert(n + 1)
    
    check, inverted = solveInverse(Hn)
    
    # compare to the true inverse
    true_inverse_Hn = la.invhilbert(n + 1)
    
    # so we do || true_inverse - own_inverse || infinity norm
    
    infinNorm = max(abs(true_inverse_Hn - inverted))
    

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

## (d)
It is known that the $2$-condition number $cond_2(H_n)$ of the Hilbert matrix grows like $\mathcal O\left(\frac{(1+\sqrt{2})^{4n}}{\sqrt{n}}\right)$. Does the $\infty$-condition number (defined in Example 2.5) of $H_n$ grow in a similar way?
For the matrix inverse, try both your own matrix inversion routine, and `scipy.linalg.invhilbert`. Output a (`plt.semilogy`) graph showing your results.